In [27]:
#important imports
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from os import listdir
import shutil
from PIL import Image, ImageOps
from collections import deque
from collections import Counter
from tensorflow.keras.models import load_model

model = load_model('proto_model_test.h5')

#LEFT = 0 / RIGHT = 1

#setting important variables
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
threshold = 0.95

def save_landmark (landmarks, label=-1):
    lm = []
    for i in range(33):
        if i not in range(13,23):
            lm.append(landmarks[i].x)
            lm.append(landmarks[i].y)
            lm.append(landmarks[i].z)
            lm.append(landmarks[i].visibility)    
    if(label != -1): lm.append(label)
    return lm

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [28]:
from scipy.special import softmax

val_data = pd.read_csv("lunge_landmark.csv",header=None)
predict_val = []
predict_val = val_data.iloc[:,1:-1].values

predict_val = np.array(predict_val)

print(predict_val.shape)

print(np.array([predict_val[0]]).shape)

test_res0 = model.predict(np.array([predict_val[0]]))
print(softmax(test_res0, axis=1))
print(np.squeeze(test_res0))
# print(model.predict_proba(np.array([predict_val[0]])))
print(test_res0[0][0])
print(test_res0[0][1])
# print(test_res0[1])
print(np.argmax(np.squeeze(test_res0)))

test_res1 = model.predict(np.array([predict_val[1]]))
print(softmax(test_res1, axis=1))
print(np.squeeze(test_res1))
# print(model.predict_proba(np.array([predict_val[1]])))
print(np.argmax(np.squeeze(test_res1)))

(2, 92)
(1, 92)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[[0.7290593 0.2709407]]
[0.9949275  0.00507242]
0.9949275
0.0050724223
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[[0.26901364 0.73098636]]
[1.8362343e-04 9.9981636e-01]
1


In [29]:
prediction = deque(maxlen=6)
prediction.append(2)
prediction.append(2)
prediction.append(2)
prediction.append(2)
prediction.append(2)
prediction.append(2)

cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            data = save_landmark(landmarks)
            data = np.array(data, dtype='float32')
            data = np.expand_dims(data, axis=0)
            # Save relevant key joints for angle calculations
            # l_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            # r_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            l_HIP = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            r_HIP = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            l_KNEE = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            r_KNEE = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            l_ANKLE = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            r_ANKLE = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            # # print(data.shape)
            res = model.predict(data)
            final_res = np.argmax(np.squeeze(res))
            

            if res[0][np.argmax(res)] < threshold: 
                # prediction.append(str(2))
                print_res = 2
            else: 
                # prediction.append(str(final_res))
                print_res = final_res
                
            prediction.append(print_res)
            # print("Prediction addition:" + str(print_res))
            # print(prediction)
            
        else: res_text = 'Pose Not Detected'    
            # print("Res:", res[0][np.argmax(res)])
        print_res = Counter(prediction).most_common(1)[0][0]
        # print("Most common:" + str(print_res))

            
            # print(data)
            # # Get coordinates
            # shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            # elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            # wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # # Calculate angle
            # angle = calculate_angle(shoulder, elbow, wrist)
            
            # # Visualize angle
            # cv2.putText(image, str(angle), 
            #                tuple(np.multiply(elbow, [640, 480]).astype(int)), 
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )
                       
    
        if print_res == 2: res_text = 'Pose Not Detected'
        else: res_text = 'Lunge'
        
        print_text = ' '
        try:
            if print_res != 2: 
                l_leg = calculate_angle(l_HIP, l_KNEE, l_ANKLE)
                r_leg = calculate_angle(r_HIP, r_KNEE, r_ANKLE)
                # l_waist = calculate_angle(l_SHOULDER, l_HIP, l_KNEE)
                # r_waist = calculate_angle(r_SHOULDER, r_HIP, r_KNEE)
                cv2.putText(image, str(l_leg), 
                           tuple(np.multiply(l_KNEE, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, str(r_leg), 
                           tuple(np.multiply(r_KNEE, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                if print_res == 1:
                    if l_leg < 70 or l_leg > 110: print_text = 'Square Left Knee'
                    if r_leg < 120: print_text = 'Stretch Right Leg'
                    else: print_text = 'Correct Posture'
                    
                if print_res == 0:
                    if r_leg < 70 or r_leg > 110: print_text = 'Square Right Knee'
                    if l_leg < 120: print_text = 'Stretch Left Leg'
                    else: print_text = 'Correct Posture'
                    
        except: pass
        
        # print(prediction)
        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        # cv2.putText(image, str(final_res) + res_text + print_text, (3,30), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 1,
        #             (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, res_text + ' ' + print_text, (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


d:\Internship Testing\MediaPipePoseEstimation-main\MPLSTM_3.9\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

In [30]:
val_data = pd.read_csv("lunge_landmark.csv",header=None)
predict_val = []
predict_val = val_data.iloc[:,1:-1].values


predict_val = np.array(predict_val)
print(predict_val)

res = model.predict(predict_val)
print (np.argmax(res))

[[ 3.59708160e-01  3.39032948e-01  1.32435904e-04  9.94135499e-01
   3.66261244e-01  3.25931638e-01 -1.97877083e-02  9.92705762e-01
   3.67926568e-01  3.25662017e-01 -1.97835155e-02  9.91642773e-01
   3.69760960e-01  3.25352520e-01 -1.98373366e-02  9.94493306e-01
   3.66504997e-01  3.25631768e-01  1.41018610e-02  9.96573448e-01
   3.68278921e-01  3.25127423e-01  1.41463950e-02  9.95963931e-01
   3.70284110e-01  3.24534923e-01  1.41520472e-02  9.96535122e-01
   3.86533976e-01  3.30684006e-01 -7.00394660e-02  9.90005076e-01
   3.86926264e-01  3.28840166e-01  7.06672743e-02  9.95655000e-01
   3.65125477e-01  3.53493571e-01 -1.94699205e-02  9.94630456e-01
   3.65274459e-01  3.52994263e-01  2.34412663e-02  9.96192336e-01
   4.06528890e-01  4.07295138e-01 -1.56155169e-01  9.99250412e-01
   4.14753228e-01  3.99004608e-01  1.53304711e-01  9.98983860e-01
   4.06323433e-01  6.00521803e-01 -9.90252793e-02  9.96262133e-01
   4.11120325e-01  5.94424665e-01  9.89951342e-02  9.97752249e-01
   2.95537

In [31]:
# if self.is_body_detected is False:
#             return NONE_ACTION_IDX

#         # res = self.model.predict(np.expand_dims(inp_seq, axis=0), verbose=0)[0]
#         res = self.model.predict(np.expand_dims(inp_seq, axis=0))[0]

#         '''
#         # Since our model has a softmax layer at the end, the output will be
#         # an array of percentages which totals to 1:
#         # EX: [0.1 0.3 0.6]
#         # When we use res[np.argmax(res)], np.argmax(res) will take the index
#         # of the highest element in the list. With res[np.argmax(res)] we get
#         # the highest element.
#         Reason why we don't use max(res) is because we still need the index
#         to get our action string in actions[].
#         But yeah the INDEX from np.argmax(res) is basically our index. 
#         '''
#         # current_action = ""
#         if res[np.argmax(res)] > self.pred_threshold:
#             current_action_idx = np.argmax(res)
#         else:
#             current_action_idx = NONE_ACTION_IDX

#         return current_action_idx

: 